# Crawler de Dados - Chaves na Mão

Esse notebook tem o mesmo código do script simple_crawler.py.

In [ ]:
import json
import logging
from pathlib import Path
from time import sleep
from typing import Any

import requests
from pydantic import BaseModel

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging = logging.getLogger(__name__)

In [ ]:
class Anuncio(BaseModel):
    id: int
    titulo: str
    ativo: bool
    aceita_troca: bool
    pet_friendly: bool
    descricao: str
    area_total: float
    area_util: float
    categoria: str
    preco: float
    preco_fmt: str
    preco_iptu: str
    imagens: list[str]
    transacao: str
    suites: int
    quartos: int
    banheiros: int
    garagens: int
    latitude: float
    longitude: float
    rua: str
    bairro: str
    cidade: str
    estado: str
    cep: str
    data_atualizacao: str

    def endereco(self) -> str:
        partes = [self.rua, self.bairro, self.cidade, self.estado, self.cep]
        return ", ".join([parte for parte in partes if parte])

In [ ]:
base_url = "https://www.chavesnamao.com.br"
base_path = "/api/realestate/listing/items/"

In [ ]:
def _configura_ultima_pagina(payload: dict[str, Any], ultima: int) -> int:
    if ultima <= 0 or ultima > 100:
        payload.get("metadata", {}).get("totalPages", 1)
        logging.info(f"Número total de páginas: {ultima}")

        if ultima > 100:
            logging.warning("Atenção: É permitido coletar no máximo até a página 100")
            ultima = 100

    return ultima

In [ ]:
def _proxima_url(payload: dict[str, Any]) -> str:
    next_url = payload.get("metadata", {}).get("links", {}).get("nextApiParams", "")
    if next_url:
        next_url = f"{base_url}{base_path}{next_url}"

    return next_url

In [ ]:
def coletar_pagina(url: str) -> dict[str, Any]:
    payload = requests.get(url, timeout=15).json()
    return payload

In [ ]:
def parse_float(value: Any) -> float:
    if isinstance(value, (int, float)):
        return float(value)
    elif isinstance(value, str):
        try:
            return float(value.replace(",", "."))
        except ValueError:
            return 0.0
    else:
        return 0.0

In [ ]:
def extrair_count(item: dict[str, Any], prop_name: str) -> int:
    prop = item.get(prop_name, {})
    count = prop.get("count", 0)
    max = prop.get("max", 0)

    if count == 0 and max == 0:
        return 0

    if isinstance(max, int):
        return max

    if isinstance(count, int):
        return count

    if isinstance(count, str):
        try:
            return int(count)
        except ValueError:
            return 0

    return 0

In [ ]:
def extrair_coordenadas(item: dict[str, Any], prop_name: str) -> float:
    location = item.get("location", {})
    if "geoposition" not in location:
        return 0.0

    geoposition = location.get("geoposition", {})
    if prop_name not in geoposition:
        return 0.0

    coordenada = geoposition.get(prop_name, 0.0)
    if isinstance(coordenada, (int, float)):
        return float(coordenada)
    elif isinstance(coordenada, str):
        try:
            return float(coordenada)
        except ValueError:
            return 0.0
    else:
        return 0.0

In [ ]:
def extrair_localizacao(item: dict[str, Any], prop_name: str) -> str:
    location = item.get("location", {})
    if prop_name not in location:
        return ""

    prop = location.get(prop_name)

    if not isinstance(prop, dict):
        return str(prop).strip()

    if "name" not in prop:
        return ""

    nome = prop.get("name", "")
    return str(nome).strip()

In [ ]:
def extrair_dados(payload: dict[str, Any]) -> list[Anuncio]:
    anuncios: list[Anuncio] = []
    items = payload["items"]

    for item in items:
        if "id" not in item:
            continue

        anuncio = Anuncio(
            id=item.get("id", 0),
            titulo=item.get("title", ""),
            ativo=item.get("active", False),
            aceita_troca=item.get("acceptTrade", False),
            pet_friendly=item.get("petFriendly", False),
            descricao=item.get("description", ""),
            area_total=parse_float(item.get("area", {}).get("total", 0.0)),
            area_util=parse_float(item.get("area", {}).get("useful", 0.0)),
            categoria=item.get("category", ""),
            preco=item.get("prices", {}).get("rawPrice", 0.0),
            preco_fmt=item.get("prices", {}).get("main", ""),
            preco_iptu=item.get("prices", {}).get("iptuValue", ""),
            imagens=item.get("pictures", {}).get("list", []),
            transacao=item.get("transaction", ""),
            quartos=extrair_count(item, "bedrooms"),
            suites=item.get("suites", {}).get("count", 0) or 0,
            banheiros=item.get("bathrooms", {}).get("count", 0) or 0,
            garagens=item.get("garages", {}).get("count", 0) or 0,
            latitude=extrair_coordenadas(item, "lat"),
            longitude=extrair_coordenadas(item, "lon"),
            rua=extrair_localizacao(item, "street"),
            bairro=extrair_localizacao(item, "neighborhood"),
            cidade=extrair_localizacao(item, "city"),
            estado=extrair_localizacao(item, "state"),
            cep=extrair_localizacao(item, "zipCode"),
            data_atualizacao=item.get("updatedAt", ""),
        )
        anuncios.append(anuncio)

    return anuncios

In [ ]:
def salvar_anuncios_jsonl(anuncios: list[Anuncio], filename: str) -> None:
    with Path(filename).open("w", encoding="utf-8") as f:
        for anuncio in anuncios:
            f.write(anuncio.model_dump_json() + "\n")

In [ ]:
def salva_payload_json(payload: dict[str, Any], filename: str) -> None:
    with Path(filename).open("w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=4)

In [ ]:
def coletar_paginas(
    level1: str,
    level2: str,
    primeira: int = 0,
    ultima: int = -1,
) -> list[Anuncio]:
    todos_anuncios: list[Anuncio] = []

    url = f"{base_url}{base_path}?level1={level1}&level2={level2}&pg={primeira}"
    logging.info(f"Coletando página {primeira}...")
    # Coleta a primeira página
    payload = coletar_pagina(url)

    # Insere os anúncios da primeira página
    todos_anuncios.extend(extrair_dados(payload))

    # Define a última página com base no total de páginas retornado pela API
    ultima = _configura_ultima_pagina(payload, ultima)

    # Obtém a URL da próxima página
    next_url = _proxima_url(payload)

    for page in range(primeira + 1, ultima + 1):
        try:
            logging.info(f"Coletando página {page}...")
            payload = coletar_pagina(next_url)
            anuncios = extrair_dados(payload)
            todos_anuncios.extend(anuncios)

            next_url = _proxima_url(payload)
        except Exception as e:
            logging.error(f"Erro ao coletar página {page}: {e}")
            salva_payload_json(payload, f"../dados/erro_{level2}_pagina_{page}.json")
            continue
        finally:
            # Adiciona um delay de 1s entre as requisições para não sobrecarregar a API
            sleep(1)

    return todos_anuncios

In [ ]:
level1 = "imoveis-a-venda"
level2 = "go-goiania"
arquivo_de_saida = f"imoveis_{level2}.jsonl"

In [ ]:
anuncios = coletar_paginas(level1, level2, primeira=1, ultima=1)
logging.info(f"Salvando {len(anuncios)} anúncios em {arquivo_de_saida}...")
salvar_anuncios_jsonl(anuncios, arquivo_de_saida)

In [ ]:
pagina1 = f"{base_url}{base_path}?level1={level1}&level2={level2}&pg=1"
payload = coletar_pagina(pagina1)

In [ ]:
payload

In [ ]:
anuncios = extrair_dados(payload)

In [ ]:
print(anuncios[0].model_dump_json(indent=2))